# Liskov Substitution principle

The Liskov Substitution Principle (LSP) states that objects of a superclass should be replaceable with objects of a subclass without altering the correctness of the program. It ensures that a subclass behaves in such a way that it can seamlessly substitute for its parent class.

**PaymentProcessor** and its derived classes before using liskov substitution principle.


In [1]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for quantity, price in zip(self.quantities, self.prices):
            total += quantity * price
        return total


class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order, security_code):
        pass


class DebitCardPaymentProcessor(PaymentProcessor):

    def pay(self, order, security_code):
        print("Processing credit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class CreditCardPaymentProcessor(PaymentProcessor):

    def pay(self, order, security_code):
        print("Processing credit card payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def pay(self, order, security_code):
        print("Processing paypal payment")
        print(f"Verifying security code: {security_code}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = CreditCardPaymentProcessor()
processor.pay(order, "0372846")
print(order.total_price(), order.status)
order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor()
paypal_processor.pay(order, "567385")
print(order.total_price(), order.status)

Processing credit card payment
Verifying security code: 0372846
210 paid
Processing paypal payment
Verifying security code: 567385
310 paid


In our refactoring, we have introduced a new violation of *SOLID* principles. **Paypal** uses *email addresses* for verification, whereas credit and debit cards use security codes. This means we are abusing the Liskov Substitution Principle because we are using a subclass in a way that is not compatible with its parent class.

Let's refactor this code to adhere to the LSP

In [6]:
from abc import ABC, abstractmethod


class Order:
    def __init__(self):
        self.items = []
        self.quantities = []
        self.prices = []
        self.status = "open"

    def add_item(self, name: str, quantity: int, price: float) -> None:
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)

    def total_price(self):
        total = 0
        for quantity, price in zip(self.quantities, self.prices):
            total += quantity * price
        return total


class PaymentProcessor(ABC):

    @abstractmethod
    def pay(self, order):
        pass


class DebitCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class CreditCardPaymentProcessor(PaymentProcessor):

    def __init__(self, security_code):
        self.security_code = security_code

    def pay(self, order):
        print("Processing credit card payment")
        print(f"Verifying security code: {self.security_code}")
        order.status = "paid"


class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self, email_address):
        self.email_address = email_address

    def pay(self, order):
        print("Processing paypal payment")
        print(f"Verifying Email Address: {self.email_address}")
        order.status = "paid"


order = Order()

order.add_item("Keyboard", 1, 50)
order.add_item("SSD", 1, 150)
order.add_item("USB cable", 2, 5)

processor = CreditCardPaymentProcessor("0372846")
processor.pay(order)
print(order.total_price(), order.status)
order.add_item("Mouse", 1, 100)
paypal_processor = PaypalPaymentProcessor("mohit.gureja@paypal.com")
paypal_processor.pay(order)
print(order.total_price(), order.status)

Processing credit card payment
Verifying security code: 0372846
210 paid
Processing paypal payment
Verifying Email Address: mohit.gureja@paypal.com
310 paid


Now the code adheres to the LSP because we are using the subclasses in a way that is compatible with their parent class